In [0]:
import os
import cv2
import numpy as np
# os.chdir('portrait-mode/')
import predict_depth
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from deeplab_model import DeepLabModel
from monodepth_model import monodepth_parameters, MonodepthModel 

In [0]:
pretrained_deeplabv3_path = 'models/deeplabv3_pascal_train_aug'
pretrained_monodepth_path = 'models/model_city2kitti/model_city2kitti'

In [9]:
tf.reset_default_graph()

def post_process_disparity(disp):
    _, h, w = disp.shape
    l_disp = disp[0, :, :]
    r_disp = np.fliplr(disp[1, :, :])
    m_disp = 0.5 * (l_disp + r_disp)
    l, _ = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
    l_mask = 1.0 - np.clip(20 * (l - 0.05), 0, 1)
    r_mask = np.fliplr(l_mask)
    return r_mask * l_disp + l_mask * r_disp + (1.0 - l_mask - r_mask) * m_disp

INPUT_HEIGHT, INPUT_WIDTH = 512, 256
params = monodepth_parameters(
      encoder='vgg',
      height=INPUT_HEIGHT,
      width=INPUT_WIDTH,
      batch_size=2,
      num_threads=1,
      num_epochs=1,
      do_stereo=False,
      wrap_mode="border",
      use_deconv=False,
      alpha_image_loss=0,
      disp_gradient_loss_weight=0,
      lr_loss_weight=0,
      full_summary=False)

left = tf.placeholder(
    tf.float32, [2, INPUT_HEIGHT, INPUT_WIDTH, 3])

disp_model = MonodepthModel(params, "test", left, None)
config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())


def predict(input_image,model):
  INPUT_HEIGHT, INPUT_WIDTH = 512, 256

  input_image = cv2.resize(
      input_image, (INPUT_WIDTH, INPUT_HEIGHT), cv2.INTER_AREA)
  input_image = input_image.astype(np.float32) / 255
  input_images = np.stack((input_image, np.fliplr(input_image)), 0)


  disp = sess.run(model.disp_left_est[0], feed_dict={left: input_images})
  disp_pp = post_process_disparity(disp.squeeze()).astype(np.float32)
  
  return disp_pp

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
!wget https://cseweb.ucsd.edu/~weijian/static/datasets/celeba/img_align_celeba.zip
!unzip img_align_celeba.zip

In [0]:
#DeepLab Model
ss_model = DeepLabModel(pretrained_deeplabv3_path)

In [0]:
#Accessing file and performing semantic segmentation
for files in tqdm(os.listdir("./img_align_celeba/")[4400:]):
  image_path = "img_align_celeba/"+files
  print(image_path)
  orig_img = cv2.imread(image_path)
  orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
  H, W, C = orig_img.shape
  #Predicting Depth
  disp_pp = predict(orig_img, disp_model)
  disp_pp = cv2.resize(disp_pp.squeeze(), (W, H))
  disp_pp = disp_pp / disp_pp.max()  
  
  #Performing Segmentation
  seg_map = ss_model.run(orig_img)
  #Taking Background
  obj_mask = seg_map > 0

  result = orig_img.copy()
  threshs = [0.8, 0.5, 0.3]
  kernels = [5, 9, 11]
  fg_masks = [disp_pp < thresh for thresh in threshs]
  for i, fg_mask in enumerate(fg_masks):
      kernel_size = kernels[i]
      #Blurring Image
      blurred = cv2.GaussianBlur(orig_img, (kernel_size, kernel_size), 0)
      result[fg_mask] = blurred[fg_mask]
  result[obj_mask] = orig_img[obj_mask]
  plt.imsave('results/'+files,result)